In [1]:
import sys
#sys.path.append('/pl/active/banich/studies/Relevantstudies/abcd/env/lib/python3.7/site-packages')
sys.path.append('/pl/active/banich/studies/Relevantstudies/abcd/data/clustering/analysis/')
from functions import *

In [2]:
def get_resids(data, var, formula):
    import statsmodels.formula.api as smf
    model = smf.ols(var + formula, data=data)
    model = model.fit()
    resid = model.resid
    return resid

def add_resid(data, var, formula):
    data[var + "_resid"] = get_resids(data, var, formula)
    return data

full_formula = '~ interview_age + C(sex)'
no_sex_formula = '~ interview_age'

In [3]:
include = pd.read_csv('/pl/active/banich/studies/Relevantstudies/abcd/data/clustering/dont_inclusion_ids.csv')[['ID']]

include.shape

(2174, 1)

In [4]:
#participants = participants.merge(include, on='ID')

dropped_siblings = pd.read_csv('/pl/active/banich/studies/Relevantstudies/abcd/data/clustering/dropped_siblings.csv')

participants_full = (pd.read_csv('/pl/active/banich/studies/Relevantstudies/abcd/data/clustering/participants.tsv', sep='\t')
                        .assign(ID = lambda x: x['participant_id'].str.replace('sub-', ''),
                                sex = lambda x: np.where(x['sex'] == 1, 'Male', 'Female'))
                        .drop_duplicates('ID', keep='first')
                    )

#participants_site_scanner_software = participants_full[['ID', 'site', 'scanner_model', 'scanner_software']]

participants = participants_full[['ID', 'matched_group']]
participants_sex = participants_full[['ID', 'sex']]

participants = participants.merge(dropped_siblings, on='ID')
participants = participants.merge(include, on='ID')

sample1 = participants.query('matched_group == 1')
sample2 = participants.query('matched_group == 2')

males = participants_sex.query('sex == "Male"')
females = participants_sex.query('sex == "Female"')

In [5]:
rpath_subcort = '/pl/active/banich/studies/Relevantstudies/abcd/data/clustering/rest/mrirscor02.txt'


rest_baseline_subcort = readtxt(rpath_subcort)
rest_baseline_subcort['src_subject_id'] = rest_baseline_subcort['src_subject_id'].str.replace('_', '', regex=True)

rest_baseline_subcort_mask = rest_baseline_subcort['eventname'].str.contains('baseline')   
rest_baseline_subcort = rest_baseline_subcort[rest_baseline_subcort_mask]

rest_baseline_subcort = rest_baseline_subcort.drop(['collection_id', 'interview_age', 'mrirscor02_id','dataset_id', 'subjectkey',
                                                    'interview_date', 'eventname','rsfmri_cor_ngd_scs_visitid', 'sex',
                                                    'rsfmri_cor_ngd_scs_tr','rsfmri_cor_ngd_scs_numtrs','rsfmri_cor_ngd_scs_nvols',
                                                    'rsfmri_cor_ngd_scs_stnvols','rsfmri_cor_ngd_scs_stcgnvols','rsfmri_cor_ngd_scs_ntpoints',
                                                    'rsfmri_cor_ngd_scs_meanmn','rsfmri_cor_ngd_scs_maxmn','rsfmri_cor_ngd_scs_meantrans',
                                                    'rsfmri_cor_ngd_scs_maxtrans','rsfmri_cor_ngd_scs_meanrot','rsfmri_cor_ngd_scs_maxrot','collection_title'], axis = 1).rename(columns={"src_subject_id": "ID"})

rest_baseline_subcort.columns = rest_baseline_subcort.columns.str.replace('rsfmri_cor_ngd_', '', regex=True)
rest_baseline_subcort.columns = rest_baseline_subcort.columns.str.replace('_scs_', '_', regex=True) 

rest_baseline_subcort[rest_baseline_subcort.iloc[:, 1:].columns] = rest_baseline_subcort.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')

rest_baseline_subcort = pd.merge(rest_baseline_subcort, participants[['ID']], on='ID')

In [6]:
#rest_creation = '''
rpath = '/pl/active/banich/studies/Relevantstudies/abcd/data/clustering/rest/abcd_betnet02.txt'

rest_baseline = readtxt(rpath)
rest_baseline['src_subject_id'] = rest_baseline['src_subject_id'].str.replace('_', '', regex=True)

rest_1_mask = rest_baseline['rsfmri_c_ngd_visitid'].str.contains('baseline')   
rest_baseline= rest_baseline[rest_1_mask]

rest_motion = rest_baseline[['src_subject_id', 'rsfmri_c_ngd_tr', 'rsfmri_c_ngd_numtrs',
       'rsfmri_c_ngd_nvols', 'rsfmri_c_ngd_stnvols',
       'rsfmri_c_ngd_stcontignvols', 'rsfmri_c_ngd_ntpoints',
       'rsfmri_c_ngd_meanmotion', 'rsfmri_c_ngd_maxmotion',
       'rsfmri_c_ngd_meantrans', 'rsfmri_c_ngd_maxtrans',
       'rsfmri_c_ngd_meanrot', 'rsfmri_c_ngd_maxrot']].rename(columns={"src_subject_id": "ID"})

rest_baseline = rest_baseline.drop(['collection_id', 'abcd_betnet02_id', 'dataset_id', 'subjectkey','interview_date', 'eventname',
       'rsfmri_c_ngd_visitid', 'rsfmri_c_ngd_tr', 'rsfmri_c_ngd_numtrs',
       'rsfmri_c_ngd_nvols', 'rsfmri_c_ngd_stnvols',
       'rsfmri_c_ngd_stcontignvols', 'rsfmri_c_ngd_ntpoints',
       'rsfmri_c_ngd_meanmotion', 'rsfmri_c_ngd_maxmotion',
       'rsfmri_c_ngd_meantrans', 'rsfmri_c_ngd_maxtrans',
       'rsfmri_c_ngd_meanrot', 'rsfmri_c_ngd_maxrot', 'collection_title'], axis = 1).rename(columns={"src_subject_id": "ID"})

rest_baseline.columns = rest_baseline.columns.str.replace('rsfmri_c_ngd_', '', regex=True)
rest_baseline.columns = rest_baseline.columns.str.replace('_ngd_', '_', regex=True) 

rest_cols_subset = ['ID','interview_age', 'sex', 'ad_ad','ad_cgc','ad_ca','ad_dt','ad_dla',
'ad_fo','ad_n','ad_rspltp','ad_smh','ad_smm','ad_sa','ad_vta','ad_vs','cgc_cgc',
'cgc_ca','cgc_dt','cgc_dla','cgc_fo','cgc_n','cgc_rspltp','cgc_smh','cgc_smm','cgc_sa',
'cgc_vta','cgc_vs','ca_ca','ca_dt','ca_dla','ca_fo','ca_n','ca_rspltp','ca_smh','ca_smm',
'ca_sa','ca_vta','ca_vs','dt_dt','dt_dla','dt_fo','dt_n','dt_rspltp','dt_smh','dt_smm','dt_sa','dt_vta',
'dt_vs','dla_dla','dla_fo','dla_n','dla_rspltp','dla_smh','dla_smm','dla_sa','dla_vta',
'dla_vs','fo_fo','fo_n','fo_rspltp','fo_smh','fo_smm','fo_sa','fo_vta','fo_vs','n_n',
'n_rspltp','n_smh','n_smm','n_sa','n_vta','n_vs','rspltp_rspltp','rspltp_smh','rspltp_smm',
'rspltp_sa','rspltp_vta','rspltp_vs','smh_smh','smh_smm','smh_sa','smh_vta','smh_vs',
'smm_smm','smm_sa','smm_vta','smm_vs','sa_sa','sa_vta','sa_vs','vta_vta','vta_vs','vs_vs']

rest_baseline = rest_baseline[rest_cols_subset]

rest_baseline[rest_cols_subset[3:]] = rest_baseline[rest_cols_subset[3:]].apply(pd.to_numeric, errors='coerce')
rest_baseline['interview_age'] = rest_baseline['interview_age'].apply(pd.to_numeric, errors='coerce')
#rest_baseline = pd.merge(rest_baseline, participants_sex, on = 'ID').dropna().drop_duplicates()
rest_baseline = pd.merge(rest_baseline, participants[['ID']], on='ID')

In [7]:
rest_baseline = pd.merge(rest_baseline, rest_baseline_subcort, on ='ID').dropna()

rest_baseline.shape

(1343, 341)

In [8]:
rest_baseline_resid = rest_baseline.copy()

for i in rest_baseline_resid.drop(['ID', 'sex', 'interview_age'], axis=1).columns:
    rest_baseline_resid = add_resid(rest_baseline_resid, i, full_formula)

In [9]:
rest_baseline_resid = pd.concat([rest_baseline_resid[['ID']],rest_baseline_resid.filter(regex='_resid')], axis=1)
rest_baseline_resid.columns = [col.replace('_resid', '') for col in rest_baseline_resid.columns]

In [10]:
#rest_cols = rest_baseline_resid.iloc[:, 1:].columns.to_list()
#rest_outliers = get_anoms(rest_baseline_resid, rest_cols)
#print(rest_outliers.groupby('outliers_sum').count().iloc[:, :1])
#rest_outliers = rest_outliers.query('outliers_sum != -3').drop('outliers_sum', axis=1)

In [11]:
sample1_rest = pd.merge(rest_baseline_resid, sample1, on='ID').drop('matched_group', axis=1)
sample2_rest = pd.merge(rest_baseline_resid, sample2, on='ID').drop('matched_group', axis=1)
#sample1_rest= pd.merge(rest_outliers, sample1, on='ID').drop('matched_group', axis=1)
#sample2_rest= pd.merge(rest_outliers, sample2, on='ID').drop('matched_group', axis=1)

#sample1_rest.to_csv('/pl/active/banich/studies/Relevantstudies/abcd/data/clustering/subtype_data/baseline/di_sample1_rest_baseline_07282023.csv')
#sample2_rest.to_csv('/pl/active/banich/studies/Relevantstudies/abcd/data/clustering/subtype_data/baseline/di_sample2_rest_baseline_07282023.csv')

In [12]:
full_sample = pd.concat([sample1_rest, sample2_rest])

full_sample.to_csv('/pl/active/banich/studies/Relevantstudies/abcd/data/clustering/subtype_data/baseline/fullsample_rest_baseline_07282023_dont_include.csv')

In [13]:
full_sample

,ID,ad_ad,ad_cgc,ad_ca,ad_dt,ad_dla,ad_fo,ad_n,ad_rspltp,ad_smh,...,vs_vtdclh,vs_crcxrh,vs_thprh,vs_cderh,vs_ptrh,vs_plrh,vs_hprh,vs_agrh,vs_aarh,vs_vtdcrh
0,NDARINV8U909UGM,-0.035092,0.002613,0.305524,0.132976,-0.139854,-0.103502,0.022689,0.043401,-0.043870,...,0.221737,-0.364404,0.004572,0.028422,-0.022615,-0.056996,-0.113979,0.363398,0.225292,-0.092906
1,NDARINVPDFDEURE,0.042077,0.038674,0.020921,0.051487,-0.040596,0.027972,0.003105,0.023952,0.055706,...,0.127261,-0.014521,-0.058574,0.027367,-0.188169,0.092172,0.090827,0.188963,0.054035,-0.202930
2,NDARINVA4ZHJBK2,0.071629,0.106272,-0.018175,-0.033070,-0.010772,-0.028400,-0.002457,-0.051210,-0.038566,...,0.037578,-0.120017,0.029737,0.051622,-0.058020,-0.013919,0.144985,0.048525,0.006014,0.006490
3,NDARINVFRRR1UGL,0.078439,0.105829,-0.115600,-0.068982,0.013790,0.044595,-0.066614,-0.192873,0.067085,...,-0.099398,-0.133273,-0.013656,-0.052569,-0.231932,-0.116934,0.130423,0.039664,0.135795,-0.163721
4,NDARINVHD01JT5V,0.071805,0.029189,-0.027834,0.037720,-0.031075,0.060735,0.098742,0.160805,0.010906,...,-0.146612,0.086869,-0.031990,-0.013088,-0.105584,0.118035,0.112070,-0.129410,0.037658,0.085286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
657,NDARINV5XRG1L4T,0.124062,0.026761,-0.043470,-0.009784,-0.060066,-0.048449,-0.015212,-0.069426,0.030428,...,-0.064267,-0.015711,-0.009345,-0.022019,-0.021930,-0.045053,-0.063514,-0.029335,-0.045553,0.052612
658,NDARINV7A80G68A,0.038406,0.067585,-0.106890,-0.074796,0.046383,-0.012883,-0.034190,-0.130644,-0.005042,...,-0.049884,0.020597,-0.023708,0.000710,0.091417,0.015386,-0.001900,0.068989,-0.102001,0.021457
659,NDARINVFN8AVY82,-0.005012,-0.026720,0.042819,0.039774,-0.086074,0.031753,0.027522,-0.094668,-0.009389,...,-0.023637,-0.048024,0.023592,-0.008186,-0.106173,0.043041,0.010345,0.208938,0.007335,-0.066438
660,NDARINVK051J689,0.165589,0.064894,0.081792,-0.045360,-0.097724,-0.051761,-0.023949,-0.034813,0.074901,...,0.139709,0.029847,0.037127,0.003713,0.022915,-0.102289,-0.087713,0.132409,-0.054440,0.108400
